In [338]:
import requests
from bs4 import BeautifulSoup

In [339]:
r = requests.get('https://www.trustpilot.com')
print(r.status_code)
print(r.status_code == requests.codes.ok)
print(requests.codes['temporary_redirect'])
print(requests.codes.teapot)
print(requests.codes['o/'])

200
True
307
418
None


In [340]:
start = requests.get('https://www.trustpilot.com/review/www.monday.com?')
# page = requests.get('https://www.trustpilot.com/review/www.monday.com?page={}')
soup = BeautifulSoup(start.content, 'html.parser')


In [341]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [342]:
html = list(soup.children)[2]
body = list(html.children)[3]
p = list(body.children)[1]

- review card: DONE
- Username: DONE
- Rating: DONE 
- Title: DONE
- Content: DONE
- date:MISSING 
- Useful score: SKIP
- Verified/Invited: MISSING
- Number of reviews wrote by customer: DONE
- Reply: DONE

In [343]:
# review_card = soup.find_all('div', class_='review-card')
# first_review = review_card[0]
    
# first_date = first_review.find('div', class_='review-content-header__dates')
# first_date = first_review.find('time')
# first_date

# first_verified = first_review.find('div', class_='review-label-tooltip-trigger')
# print(first_verified)
# first_verified = first_review.find('div', class_='review-label-tooltip-trigger')
# if first_verified:
#     if first_verified.get_text(strip=True) == 'Invited':
#         verified_status = 'Invited'
#     elif first_verified.get_text(strip=True) == 'Reviewer contacted':
#         verified_status = 'Reviewer contacted'
#     else:
#         verified_status = 'other'
# else:
#     verified_status = None
# print(verified_status)

In [349]:
names = []
ratings = []
titles = []
contents = []
rev_wrote = []
replies = []
PAGES = 20
COMPANY ='www.monday.com'

for p in range(1, PAGES):
    page_url = requests.get('https://www.trustpilot.com/review/' + COMPANY + '?page=' + str(p))
    soup = BeautifulSoup(page_url.content, 'html.parser')
    review_card = soup.find_all('div', class_='review-card')
    for review in review_card:
        name = review.find('div', class_='consumer-information__name').get_text(strip=True)
        names.append(name)
        rating = review.find('img').attrs.get('alt')
        ratings.append(rating)
        title = review.find('a', class_='link link--large link--dark').get_text()
        titles.append(title)
        if review.find('p', class_='review-content__text'):
            content = review.find('p', class_='review-content__text').get_text(strip=True)
            
        else:
            content = None
        contents.append(content)
        rev_written = review.span.get_text()
        rev_wrote.append(rev_written)
        reply = review.find('div', class_='review__company-reply')
        if reply:
            replies.append(True)
        else:
            replies.append(False)
    

In [350]:
import pandas as pd
test_df = pd.DataFrame({'names': names,
                       'ratings': ratings,
                       'titles': titles,
                       'contents': contents,
                       'rev_wrote': rev_wrote,
                       'replies': replies})
print(test_df.info())
print(test_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   names      380 non-null    object
 1   ratings    380 non-null    object
 2   titles     380 non-null    object
 3   contents   366 non-null    object
 4   rev_wrote  380 non-null    object
 5   replies    380 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 15.3+ KB
None
             names             ratings                                 titles  \
0            Igaln       2 stars: Poor              Too expensive too quickly   
1  Thomas O'Rourke         1 star: Bad        Way overpriced - look elsewhere   
2     Jamie Serafi  5 stars: Excellent                          Great company   
3      Lisa Kernow       2 stars: Poor              I USED to love Monday.com   
4     Daniel Ehlen         1 star: Bad  I've reset our password to the CRM 3…   

                                          

In [346]:
test_df.to_csv('reviews.csv')